In [ ]:
import os
import random
from math import exp as exp
import numpy as np
import matplotlib.pyplot as plt

import albumentations as A

import torch
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms, datasets, utils
from torchvision.datasets import ImageFolder
# from models import resnet_model

from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image


In [ ]:
# Hyperparameters & etc.

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

In [ ]:
# Dataset & Loader config

EvalDataset = ImageFolder(root = '/eval_set',
                            transform=transforms.Compose([
                                transforms.Resize((224,224)),
                                transforms.ToTensor(),
                           ]))
EvalDataLoader = DataLoader(EvalDataset, batch_size = 32, shuffle=True)

# EvalTransForms = A.Compose([    
#     A.pytorch.transforms.ToTensorV2()
# ])
# EvalDataset = Dataset('', EvalTransForms)
# EvalDataLoader = DataLoader(EvalDataset, batch_size = 4, shuffle=True)

In [ ]:
# display sample images
def Display_Sample_Images(Dataset, labels=True, nrow=4, grid_size=4, padding=2, normalize=True):

    grid_size=grid_size
    
    image_indices = np.random.randint(0,len(Dataset),grid_size)
    
    images=[Dataset[i][0] for i in image_indices]
    
    images=utils.make_grid(images, nrow=nrow, padding=padding, normalize = normalize)
    
    plt.figure(figsize=(10,10))
    npimg = images.numpy()
    npimg_tr = np.transpose(npimg, (1, 2, 0))
    plt.imshow(npimg_tr)
    
    if labels is True:
        labels=[Dataset[i][1] for i in image_indices]
        plt.title('labels: ' + str(labels))
        
    print("image indices:",image_indices)
    print(Dataset[0][0].shape)
    

In [ ]:
# Model config 
num_classes = 2
model = resnet_model(num_classes,pretrained = True)
dicts = torch.load('/.pth')

model.load_state_dict(dicts['net_state_dict'])
epoch = dicts['epoch']
acc = dicts['acc']

model = model.to(device)
print('Model Epoch : ',epoch)
print('Model Accuracy : ',acc)

In [ ]:
##########################  evaluate net and save model  ###############################

model.eval()

LogitArr=[]
LabelArr =[]
PredArr = []
EvalTotal = 0
EvalCorrect = 0
for batch_idx, (img, label) in enumerate(EvalDataLoader):
    with torch.no_grad():
        img, label = img.to(device), label.to(device)
        TotalSize = len(EvalDataLoader.dataset)
        total_batch = len(EvalDataLoader)
        batch_size = len(label)
        
        Logits = model(imgs=img)
        logit = Logits.to('cpu').detach().numpy()
        for i in range(len(logit)):
            LogitArr.append(logit[i][1])
            # LogitArr.append(logit[i][logit[i].argmax()])
            PredArr.append(np.argmax(logit[i],axis=0))
            
        labels = label.to('cpu').detach().numpy()
        for i in range(len(labels)):
            LabelArr.append(labels[i]) 
        
        # calculate accuracy
        
        EvalTotal += batch_size
        EvalCorrect += (torch.max(Logits, 1)[1]==label.data).sum().item()

In [ ]:
# AUC계산
 
def Calculate_AUC(y_true, y_score, thresholds = False ):
    fprs , tprs , thresholds = roc_curve(np.array(y_true), np.array(y_score))
    roauc = roc_auc_score(y_true, y_score)
    
    plt.plot(fprs, tprs, color='red', label='ROC')
    plt.plot([0, 1], [0, 1], color='green', linestyle='--', label='Random')
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    plt.yticks(np.round(np.arange(start, end, 0.1),2))
    plt.xlim(0,1)
    plt.ylim(0,1)
    plt.xlabel('False Positive Rate (1 - Sensitivity)')
    plt.ylabel('True Positive Rate (Recall)')
    plt.title('Receiver Operating Characteristic Curve')
    plt.legend()
    plt.show()
    print('AUC: {:.4f}'.format(roauc))
    if thresholds is True:
        print('Thresholds: ', thresholds)

In [ ]:
# Metrics 계산

def Get_Eval_Metrics(y_true, y_pred, average = 'binary', mode = 'BC', pos_label = 1):
    """
    -average종류-
    'binary': 이진분류 지정한 클래스에 대한 결과만 보고함( pos_label default=1)
    'micro': 전체 평균으로 모든 열에서 맞은 것 즉, 대각선 성분의 총 합을 총 갯수로 나눈 것 (개수 그자체로 평균),  데이터 불균형일때 조금더 효과적 
    'macro': average를 None으로 두었을 때 구한 각 열의 Precision들을 산술 평균한 값이 macro가 된다 (평균의 평균을 내는 방법). (normal+abnormal)/2*precision or recall or f1 score
    'weighted': 각 레이블에 대한 메트릭을 계산하고 지원에 따라 가중치가 부여된 평균(각 레이블에 대한 실제 인스턴스 수)을 찾는다. normal/(normal+abnormal)*precision or recall or f1 score
    'samples': 각 인스턴스에 대한 지표를 계산하고 평균을 찾음, accuracy_score와 다른 다중 레이블 분류에만 의미가 있다
    
    """
    if mode is 'ML':
        average = 'samples'
    if mode is 'MC':
        average = None
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average=average, pos_label = pos_label)
    recall = recall_score(y_true, y_pred, average=average, pos_label = pos_label)
    F1 = f1_score(y_true, y_pred, average=average, pos_label = pos_label)
    ClassificationReport = classification_report(y_true, y_pred)
    if average is None:
            print('Accuracy (Exact Match Ratio): ', accuracy)
            print('Precision: ', precision)
            print('Recall: ', recall)
            print('F1: ', F1)
            print('Classification Report: \n', ClassificationReport)
            print('예측값 10개만 표시: ', y_pred[:10])
    else:
        print('Accuracy (Exact Match Ratio): {:.4f}'.format(accuracy))
        print('Precision: {:.4f}'.format(precision))
        print('Recall: {:.4f}'.format(recall))
        print('F1: {:.4f}'.format(F1))
        print('Classification Report: \n', ClassificationReport)
        print('예측값 10개만 표시: ', y_pred[:10])
    

In [ ]:
def Confusion_Matrix(y_true, y_pred, title='Confusion matrix', cmap='Blues',normalize= None):
    confusion = confusion_matrix(y_true, y_pred)
    ConfusionMatrixDisplay.from_predictions(y_true, y_pred,cmap=cmap,normalize = normalize )
    plt.title(title)
    plt.show()
    print('Confusion matrix:\n', confusion)
    

In [ ]:
# CAM 설치 
# !pip install grad-cam

fn_tonumpy = lambda x : x.to('cpu').detach().numpy().transpose(0,2,3,1) # numpy 로 변환

def SimpleCam(model, DataLoader, target_layers, use_cuda=True):
    for i, (img, label) in enumerate(DataLoader):   
        img, label = img.to(device), label.to(device)
        input_tensor = img

        # input_tensor는 배치랑 이미지 상관없이 넣을수 있다.

        cam = GradCAM(model = model, target_layers = target_layers, use_cuda = use_cuda)

        # targets = 어떤클래스를 보고있는지 확인
        targets = None
        # targets = [ClassifierOutputTarget(1)]
        # targets = [ClassifierOutputTarget(0)]
        
        grayscale_cam = cam(input_tensor=input_tensor, targets=targets) # aug_smooth=True 와 eigen_smooth=True 로 CAM의 결과물 smoothing 가능
        grayscale_cam = grayscale_cam[0, :]
        np_arr = fn_tonumpy(img)
        np_arr = np_arr[0,:]
        visualization = show_cam_on_image(np_arr, grayscale_cam, use_rgb=True)
        fig = plt.figure()
        rows = 1
        cols = 2
        ax1 = fig.add_subplot(rows, cols, 1)
        ax1.axis("off")
        plt.imshow(np_arr)
        ax2 = fig.add_subplot(rows, cols, 2)
        plt.imshow(visualization)
        ax2.axis("off")
        plt.show() 

In [ ]:
Get_Eval_Metrics(LabelArr,PredArr)

Calculate_AUC(LabelArr, LogitArr)

Display_Sample_Images(EvalDataset)

# pip install -U scikit-learn
Confusion_Matrix(LabelArr,PredArr)

In [ ]:
SimpleCam(model, EvalDataLoader,[model.model.layer4])